# Assignment 1 : Decision Tress using `scikit-learn`

Scikit-learn provides a range of supervised and unsupervised learning algorithms via a consistent interface in Python. In this assigment you'll explore how to train decision trees using the `scikit-learn` library. The scikit-learn documentation can be found [here](http://scikit-learn.org/stable/documentation.html).

In this assignment we'll attempt to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the Diabetic Retinopathy data set, which contains 1151 instances and 20 attributes (some categorical, some continuous). You can find additional details about the dataset [here](http://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set).

In [174]:
# You may add additional imports
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import time

%matplotlib inline

In [3]:
# Read the data from csv file
col_names = []
for i in range(20):
    if i == 0:
        col_names.append('quality')
    if i == 1:
        col_names.append('prescreen')
    if i >= 2 and i <= 7:
        col_names.append('ma' + str(i))
    if i >= 8 and i <= 15:
        col_names.append('exudate' + str(i))
    if i == 16:
        col_names.append('euDist')
    if i == 17:
        col_names.append('diameter')
    if i == 18:
        col_names.append('amfm_class')
    if i == 19:
        col_names.append('label')

data = pd.read_csv("messidor_features.txt", names = col_names)
print(data.shape)
data.head(10)

(1151, 20)


quality  prescreen  ma2  ma3  ma4  ma5  ma6  ma7   exudate8   exudate9  \
0        1          1   22   22   22   19   18   14  49.895756  17.775994   
1        1          1   24   24   22   18   16   13  57.709936  23.799994   
2        1          1   62   60   59   54   47   33  55.831441  27.993933   
3        1          1   55   53   53   50   43   31  40.467228  18.445954   
4        1          1   44   44   44   41   39   27  18.026254   8.570709   
5        1          1   44   43   41   41   37   29  28.356400   6.935636   
6        1          0   29   29   29   27   25   16  15.448398   9.113819   
7        1          1    6    6    6    6    2    1  20.679649   9.497786   
8        1          1   22   21   18   15   13   10  66.691933  23.545543   
9        1          1   79   75   73   71   64   47  22.141784  10.054384   

   exudate10  exudate11  exudate12  exudate13  exudate14  exudate15    euDist  \
0   5.270920   0.771761   0.018632   0.006864   0.003923   0.003923  0.486903   
1   3.325423   0.234185   0.003903   0.003903   0.003903   0.003903  0.520908   
2  12.687485   4.852282   1.393889   0.373252   0.041817   0.007744  0.530904   
3   9.118901   3.079428   0.840261   0.272434   0.007653   0.001531  0.483284   
4   0.410381   0.000000   0.000000   0.000000   0.000000   0.000000  0.475935   
5   2.305771   0.323724   0.000000   0.000000   0.000000   0.000000  0.502831   
6   1.633493   0.000000   0.000000   0.000000   0.000000   0.000000  0.541743   
7   1.223660   0.150382   0.000000   0.000000   0.000000   0.000000  0.576318   
8   6.151117   0.496372   0.000000   0.000000   0.000000   0.000000  0.500073   
9   0.874633   0.099780   0.023386   0.000000   0.000000   0.000000  0.560959   

   diameter  amfm_class  label  
0  0.100025           1      0  
1  0.144414           0      0  
2  0.128548           0      1  
3  0.114790           0      0  
4  0.123572           0      1  
5  0.126741           0      1  
6  0.139575           0      1  
7  0.071071           1      0  
8  0.116793           0      1  
9  0.109134           0      1

### 1. Data preprocessing  & dimensionality reduction with PCA

Q1. Separate the feature columns from the class label column. You should end up with two separate data frames - one that contains all of the feature values and one that contains the class labels.

In [21]:
x_nonstd = data.drop(['prescreen'], axis=1)
y = data['prescreen']
print(x.shape, y.shape)

(1151, 19) (1151,)


Q2. Use `sklearn.preprocessing.StandardScaler` to standardize the dataset’s features (mean = 0 and variance = 1). 

Only standardize the the features, not the class labels! This will be required for running the principal component analysis (PCA) dimensionality reduction later. Note that `StandardScaler` returns a numpy array.

In [22]:
scaler = StandardScaler()
x_data = scaler.fit_transform(x_nonstd)
x = pd.DataFrame(data=x_data, index=x.index, columns=x.columns)
x.head()

quality       ma2       ma3       ma4       ma5       ma6       ma7  \
0  0.059054 -0.641486 -0.618782 -0.576463 -0.630029 -0.551116 -0.473745   
1  0.059054 -0.563391 -0.535778 -0.576463 -0.677410 -0.653676 -0.539992   
2  0.059054  0.920417  0.958299  1.046665  1.028299  0.936006  0.784951   
3  0.059054  0.647084  0.667784  0.783456  0.838776  0.730886  0.652456   
4  0.059054  0.217561  0.294265  0.388641  0.412349  0.525766  0.387468   

   exudate8  exudate9  exudate10  exudate11  exudate12  exudate13  exudate14  \
0 -0.242917 -0.246003  -0.296966  -0.271509  -0.218324  -0.194409  -0.205124   
1 -0.109250  0.032972  -0.465224  -0.408593  -0.224256  -0.197212  -0.205175   
2 -0.141383  0.227196   0.344463   0.769037   0.335538   0.152330  -0.110043   
3 -0.404199 -0.214977   0.035830   0.316953   0.112573   0.056919  -0.195765   
4 -0.788069 -0.672306  -0.717335  -0.468311  -0.225828  -0.200905  -0.214968   

   exudate15    euDist  diameter  amfm_class     label  
0  -0.186169 -1.294763 -0.468656    1.405048 -1.063711  
1  -0.186281 -0.082168  2.006054   -0.711719 -1.063711  
2  -0.164808  0.274283  1.121516   -0.711719  0.940105  
3  -0.199541 -1.423814  0.354501   -0.711719 -1.063711  
4  -0.208100 -1.685874  0.844102   -0.711719  0.940105

Q3 . Split your dataset into training and test sets (80% - 20% split). Use `sklearn.model_selection.train_test_split` to help you in this task. You should be working with your standardized features from here forward. Display how many records are in the training set and how many are in the test set.

In [48]:
# your code goes here
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
print(x_train.shape)
print(y_train.shape)

(920, 19)
(920,)


Q4. PCA is affected by the scale of the features that is why it is important to standardize the dataset first. The principle components generated by PCA are sensitive to the shape of the data in d-dimensional space. 
* Carry out a principal components analysis using `sklearn.decomposition.PCA` Note that you are fitting PCA on the **training set** only (NOT the test set).
* Use the `pca.explained_variance_ratio_` field to determine how many principal components are needed so that 95% variance is retained. 
* Reduce the PCA-transformed-dataset to this number of columns and you'll use the resulting dataset for subsequent tasks.

* Once the training set's dimensionality has been reduced with PCA, transform the **test set** to the principal component space that was created. (Do not fit a new PCA. Use the same one that was created with the training set.)

In [49]:
# run pca on train set
pca = PCA(n_components = 0.95)
pca.fit(x_train)
x_train = pca.transform(x_train)
print(x_train.shape)

# apply to test set
x_test = pca.transform(x_test)
print(x_test.shape)

(920, 9)
(231, 9)


### 2. Training Decision Trees in `scikit-learn`

Q5. Use `sklearn.tree.DecisionTreeClassifier` to fit a decision tree classifier on the training set. Use entropy as the split criterion. 

In [130]:
tree = DecisionTreeClassifier(criterion='entropy')
tree.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

Q6. Now that the tree has been learned from the training data, we can run the test data through and predict classes for our test data. Use the `predict` method of `DecisionTreeClassifier` to classify the test data. Then use `sklearn.metrics.accuracy_score` to print out the accuracy of the classifier on the test set.

In [131]:
def score_tree(tree):
    y_pred = tree.predict(x_test)
    print( accuracy_score(y_test, y_pred) )

score_tree(tree)

0.8658008658008658


Q7. Note that the DecisionTree classifier has many parameters that can be set. Try tweaking parameters like split criterion, max_depth, min_impurity_decrease, min_samples_leaf, min_samples_split, etc. to see how they affect accuracy.

In [195]:
# test w/ gini algorithm
tree = DecisionTreeClassifier(criterion='gini')
tree.fit(x_train, y_train)
score_tree(tree)
# score nearly the same, abt 1-2% less on avg

# test w/ min samples of 20 per leaf
tree = DecisionTreeClassifier(min_samples_leaf=20, criterion='entropy')
tree.fit(x_train, y_train)
score_tree(tree)
# improved the score a little (by ~4%)

# test only splitting on 1 feature at a time
tree = DecisionTreeClassifier(max_features=1, criterion='entropy')
tree.fit(x_train, y_train)
score_tree(tree)
# score goes down by abt ~3%

0.8398268398268398
0.9090909090909091
0.8658008658008658


### 3. Using K-fold Cross Validation

You have now built a decision tree and tested it's accuracy using the "holdout" method. But as discussed in class, this is not sufficient for estimating generalization accuracy. Instead, we should use Cross Validation to get a better estimate of accuracy. You will do that next.

Q8. Use `sklearn.model_selection.cross_val_score` to perform 10-fold cross validation on your decision tree. Report the accuracy of the model. For this step, revert back to using the original data before it got scaled and underwent PCA. So you want to use the data (features and labels) that came out of Q1. 

In [201]:
tree = DecisionTreeClassifier(criterion='entropy')

cv = cross_val_score(tree, x_nonstd, y, cv=10)
cv

array([0.84482759, 0.81896552, 0.82758621, 0.84482759, 0.87826087,
       0.86956522, 0.85217391, 0.85964912, 0.85964912, 0.85964912])

Q9. Now we want to tune our model to use the best parameters to avoid overfitting to our training data. Grid search is an approach to parameter tuning that will methodically build and evaluate a model for each combination of algorithm parameters specified in a grid. 
* Use `sklearn.model_selection.GridSearchCV` to find the best `max_depth`, `max_features`, and `min_samples_leaf` for your tree. Use 'accuracy' for the scoring criteria.
* Try the values [5,10,15,20] for `max_depth` and `min_samples_leaf`. Try [5,10,15] for `max_features`. 
* Use a 5-fold cross validation. 
* Print out the best value for each of the tested parameters.
* Print out the accuracy of the model with these best values.

In [208]:
tree = DecisionTreeClassifier(criterion='entropy')

param_grid = {
    'max_depth': [5, 10, 15, 20],
    'min_samples_leaf': [5, 10, 15, 20],
    'max_features': [5, 10, 15]
}
gridcv = GridSearchCV(tree, param_grid)
gridcv.fit(x_nonstd, y)
print(gridcv.best_params_)
gridcv.best_score_

{'max_depth': 5, 'max_features': 5, 'min_samples_leaf': 15}


0.9183318853171155

Q10. To perform the nested cross-validation that we discussed in class, you'll now need to pass the `GridSearchCV` into a `cross_val_score`. 

What this does is: the `cross_val_score` splits the data in to train and test sets for the first fold, and it passes the train set into `GridSearchCV`. `GridSearchCV` then splits that set into train and validation sets for k number of folds (the inner CV loop). The hyper-parameters for which the average score over all inner iterations is best, is reported as the `best_params_`, `best_score_`, and `best_estimator_`(best decision tree). This best decision tree is then evaluated with the test set from the `cross_val_score` (the outer CV loop). And this whole thing is repeated for the remaining k folds of the `cross_val_score` (the outer CV loop). 

That is a lot of explanation for a very complex (but IMPORTANT) process, which can all be performed with a single line of code!

Be patient for this one to run. The nested cross-validation loop can take some time. An asterisk [\*] next to the cell indicates that it is still running.

Print the accuracy of your tuned, cross-validated model. This is the official accuracy that you would report for your model.

In [209]:
# your code goes here
tree = DecisionTreeClassifier(criterion='entropy')

gridcv = GridSearchCV(tree, param_grid)
cv = cross_val_score(gridcv, x_nonstd, y, cv=10)
cv

array([0.9137931 , 0.9137931 , 0.9137931 , 0.9137931 , 0.92173913,
       0.92173913, 0.92173913, 0.92105263, 0.92105263, 0.92105263])

Q11. Our accuracy rate isn't very good. We wouldn't want to use this model in the real world to actually diagnose patients because it is wrong about 40% of the time! What could you do to improve the accuracy of the model? Answer as a comment:

In [ ]:
'''
As observed in Q7, enforcing a minimum number of samples per leaf improved the overall
accuracy. This suggests that the decision tree is overfitted to the data. Thus, implementing
a random forest algorithm instead of a single decision tree could minimize the damage
done by overfitting and would likely improve accuracy.
'''